# 10 Word2Vec Implemented on Keras
keras is a quite high-level deep learning library. In this notebook, we are going to implement two word2vec models: CBoW and Skip-gram. 


## Agenda

1. How to load pre-trained word vectors
2. Reading in the IMDB Sentiment Dataset and Iterating over files in Python
3. Build Skip-gram Model
4. Build CBoW Model
5. Memory-friendly Data Generation Methods

## Part 1: Load pre-trained word vectors

- You can find the word2vec project here: https://code.google.com/archive/p/word2vec/
- Download the word embeddings from the section **Pre-trained word and phrase vectors**. It is named `GoogleNews-vectors-negative300.bin.gz (3.4G)`
- Use gensim that you can easily load these wordvectors and utilize their functions

In [1]:
from gensim.models import KeyedVectors
# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

dog = model['dog']
print(dog.shape)
print(dog[:10])

# Some predefined functions that show content related information for given words
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

print(model.doesnt_match("breakfast cereal dinner lunch".split()))

print(model.similarity('woman', 'man'))

(300,)
[ 0.05126953 -0.02233887 -0.17285156  0.16113281 -0.08447266  0.05737305
  0.05859375 -0.08251953 -0.01538086 -0.06347656]
[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431607246399), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.5181134343147278), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454)]
cereal
0.76640123


In [2]:
# clear the memory
del model

## Part 2: Toy Corpus


In [3]:
import numpy as np
import pandas as pd
import os
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


#### Text tokenization from Keras

This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...

- In the following, we are going to use a toy corpus for a quick demo.

In [4]:
# let us check the texts_to_sequences function
toy_corpus = ['king is a strong man', 
              'queen is a wise woman', 
              'boy is a young man',
              'girl is a young woman',
              'prince is a young king',
              'princess is a young queen',
               'man is strong', 
               'woman is pretty',
               'prince is a boy will be king',
               'princess is a girl will be queen']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(toy_corpus)
toy_seq_corpus = tokenizer.texts_to_sequences(toy_corpus)
print(toy_seq_corpus[0])
print(tokenizer.word_index['king'])
print(tokenizer.word_index['is'])
print(tokenizer.word_index['a'])
print(tokenizer.word_index['strong'])

[4, 1, 2, 8, 5]
4
1
2
8


#### Raw Text Cleaning

In [5]:
print(tokenizer.index_word[1])

is


In [6]:
print(tokenizer.index_word[0])

KeyError: 0

- The KeyError means that the Tokenizer reserves 0 as an OOV words.
- In practive, the first ebmedding in word embedding martix is for unkown words or chars.

## Part 3: Build Skip-gram Model

- Here, we only use toy corpus for demo purpose.
- Target: predict the nearby words based on the center word.
<img src="imgs\word2vec-skip-gram.png" alt="cbow"
	title="cbow pic" width="250" height="150" />

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape
from keras.utils import to_categorical
from keras.preprocessing import sequence
import keras.backend as K

- **For skip-gram,  training data generation**:

the input x is the center word index, the output x is one hot vector of the neary word index.
For example, the toy corpus only contain two sentences.
```
I like apple 
I like reading books
```
1. The first step: build a vocab. which can be regarded as a mapping from words to interget index.

Here, OOV-> 0, I -> 1, like -> 2, apple -> 3, reading -> 4, books -> 5.

2. Then, we scan the corpus and creat the pair of center word and nearby word. Here, we set the window size is `one`.
We have the following pair of input x and target y.

<pre>
words pair              numerical input       numerical output

(I, like)                       1               [0,0,1,0,0,0]

(like, I)                       2               [0,1,0,0,0,0]

(like, apple)                   2               [0,0,0,1,0,0]

(apple, like)                   4               [0,0,1,0,0,0]

(I, like)

(like, I)

(like, reading)              

(reading, like)

(reading, books)

(books, reading)                 5              [0,0,0,0,1,0]
</pre>

In [8]:
def generate_data(corpus, window_size, V):
    """
    corpus is the collection of lists of words index
    window_size is the context size that defines 'nearby' words
    V is the vocab Size
    """
    labels = []
    in_words   = [] 
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            s = index - window_size
            e = index + window_size + 1
            for i in range(s, e):
                if 0<= i < L and i != index:
                    in_words.append([word])
                    labels.append(to_categorical(words[i], V))
    return (in_words, labels)   

In [9]:
# plus one is for OOV words
V = len(tokenizer.word_index) + 1
dim = 5
window_size = 4
ith = 0
input_x, target_y =  generate_data(toy_seq_corpus, window_size, V)
input_x           = np.array(input_x,dtype=np.int32)
target_y          = np.array(target_y,dtype=np.int32)

In [10]:
print('check the first pair of input and output')
print(input_x[0])
print(target_y[0])
print('check the third pair of input and output')
print(input_x[2])
print(target_y[2])

check the first pair of input and output
[4]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
check the third pair of input and output
[4]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


In [11]:
print(toy_seq_corpus[0])

[4, 1, 2, 8, 5]


- **Model Config**

It consists of two layers:

1. The first layer is embeddings layer, which perform the lookup operation. Given the word index as the input, the layer output will return the corresponding vector

2. The second layer is softmax layer.

- **Embeddings Layer**:

Turns positive integers (indexes) into dense vectors of fixed size. eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

This layer can only be used as the first layer in a model.

1. input_dim: int > 0. Size of the vocabulary, i.e. maximum integer index + 1.
2. output_dim: int >= 0. Dimension of the dense embedding.
3. embeddings_initializer: Initializer for the embeddings matrix (see initializers).
4. input_length: Length of input sequences, when it is constant. This argument is required if you are going to connect  Flatten then Dense layers upstream (without it, the shape of the dense outputs cannot be computed).

### Model Building

In [12]:
skipgram = Sequential()
skipgram.add(Embedding(input_dim=V, output_dim=dim, init='glorot_uniform', input_length=1))
skipgram.add(Reshape((dim, )))
skipgram.add(Dense(input_dim=dim, output_dim=V, activation='softmax'))

/Users/rui/pyenv/bt5153/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=17, output_dim=5, input_length=1, embeddings_initializer="glorot_uniform")`
  
/Users/rui/pyenv/bt5153/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=5, activation="softmax", units=17)`
  after removing the cwd from sys.path.


In [13]:
print(skipgram.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 5)              85        
_________________________________________________________________
reshape_1 (Reshape)          (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                102       
Total params: 187
Trainable params: 187
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
skipgram.compile(loss='categorical_crossentropy', optimizer="adadelta")

In [15]:
skipgram.fit(input_x, target_y, batch_size=8, epochs=30)

/Users/rui/pyenv/bt5153/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/30
204/204 [==============================] - 0s 1ms/step - loss: 2.8350
Epoch 2/30
204/204 [==============================] - 0s 229us/step - loss: 2.8209
Epoch 3/30
204/204 [==============================] - 0s 188us/step - loss: 2.8076
Epoch 4/30
204/204 [==============================] - 0s 205us/step - loss: 2.7948
Epoch 5/30
204/204 [==============================] - 0s 174us/step - loss: 2.7827
Epoch 6/30
204/204 [==============================] - 0s 161us/step - loss: 2.7712
Epoch 7/30
204/204 [==============================] - 0s 216us/step - loss: 2.7603
Epoch 8/30
204/204 [==============================] - 0s 192us/step - loss: 2.7497
Epoch 9/30
204/204 [==============================] - 0s 213us/step - loss: 2.7394
Epoch 10/30
204/204 [==============================] - 0s 176us/step - loss: 2.7295
Epoch 11/30
204/204 [==============================] - 0s 216us/step - loss: 2.7199
Epoch 12/30
204/204 [==============================] - 0s 171us/step - loss: 2.7110
Epo

- **How to save the learned word vectors**

In [16]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

5

In [17]:
vectors = skipgram.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

- **the saved format for word vectors**
<img src="imgs\saved_format.jpg" alt="cbow"
	title="saved format" width="550" height="450" />

- **we can use gensim**

Gensim is a production-ready open-source library for NLP problems.

https://radimrehurek.com/gensim/index.html


In [18]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [19]:
w2v.most_similar(positive=['man'])

[('boy', 0.6419950127601624),
 ('strong', 0.6150541305541992),
 ('princess', 0.5107337236404419),
 ('king', 0.20977374911308289),
 ('a', 0.17498812079429626),
 ('queen', 0.14768481254577637),
 ('girl', 0.02330883964896202),
 ('is', -0.016304761171340942),
 ('young', -0.04026016592979431),
 ('woman', -0.20451711118221283)]

## Part 4: Build CBoW Model

- CBoW's target is the prediction of center word.
<img src="imgs\word2vec-cbow.png" alt="cbow"
	title="cbow pic" width="250" height="150" />

- **For cbow,  training data generation**:

the input x is the list of  context word index, the output x is one hot vector of the center word.
For example, the toy corpus only contain two sentences.
```
I like apple 
I like reading books
```
1. The first step: build a vocab. which can be regarded as a mapping from words to interget index. 
Here OOV->0, I -> 1, like -> 2, apple -> 3, reading -> 4 books -> 5.

2. Then, we scan the corpus and creat the pair of list of nearby word and center word. Here, we set the window size is `one`.
We have the following pair of input x and target y.

<pre>
words pair                     numerical input       numerical output

([like], I)                        [2]                 [0,1,0,0,0,0]

([I, apple], like)                 [1,3]               [0,0,1,0,0,0]

([like], apple)                    [2]                 [0,0,0,1,0,0]

([like], I)                        [2]                 [0,1,0,0,0,0]
 
([I, reading], like)               [1,4]               [0,1,0,0,0,0]

([like, books], reading)           [2,5]               [0,0,0,0,1,0]

([reading], books)                 [4]                 [0,0,0,0,0,1]
</pre>

3. At last, sometimes, we can not get the input context with enough length. For example, the first pair's numerical input only has one word index insetad of two. What we can do here is padding the short input so that all input data have the same length. 

- **Prepare the training and labels**

In [20]:
from keras.preprocessing import sequence
import keras.backend as K
def generate_data(corpus, window_size, V):
    """
    corpus is the list of sequence of words index
    window_size is used to define  
    V is the vocab Size
    """
    context_words   = []
    center_words    = []
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)           
            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = to_categorical(labels, V)
            context_words.append(x)
            center_words.append(y)
    return context_words, center_words

In [21]:
ith = 0
input_x, target_y = generate_data(toy_seq_corpus, window_size, V)

In [22]:
input_x = np.array(input_x)
print(input_x.shape)
input_x = np.squeeze(input_x)  # sequeeze the second dimesion as on
print(input_x.shape)
target_y = np.array(target_y)
target_y = np.squeeze(target_y)

(50, 1, 8)
(50, 8)


In [23]:
print(input_x[0])
print(target_y[0])

[0 0 0 0 1 2 8 5]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [24]:
print(toy_seq_corpus[0])

[4, 1, 2, 8, 5]


In [25]:
cbow = Sequential()

- **Lambda Layer**:

Wraps arbitrary expression as a Layer object.
    1. function: The function to be evaluated. Takes input tensor as first argument. usually based on backend
    2. output_shape: Expected output shape from function. 

In [26]:
from keras.layers import Lambda
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
# sum all embeddings 
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
## add softmax layer
cbow.add(Dense(V, activation='softmax'))

In [27]:
print(cbow.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 8, 5)              85        
_________________________________________________________________
lambda_1 (Lambda)            (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 17)                102       
Total params: 187
Trainable params: 187
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
cbow.compile(loss='categorical_crossentropy', optimizer='adam')
# Train the model, iterating on the data in batches of 512 samples
cbow.fit(input_x, target_y, batch_size=8, epochs=10)

/Users/rui/pyenv/bt5153/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
50/50 [==============================] - 0s 2ms/step - loss: 2.8297
Epoch 2/10
50/50 [==============================] - 0s 318us/step - loss: 2.8261
Epoch 3/10
50/50 [==============================] - 0s 390us/step - loss: 2.8229
Epoch 4/10
50/50 [==============================] - 0s 410us/step - loss: 2.8194
Epoch 5/10
50/50 [==============================] - 0s 375us/step - loss: 2.8168
Epoch 6/10
50/50 [==============================] - 0s 288us/step - loss: 2.8132
Epoch 7/10
50/50 [==============================] - 0s 284us/step - loss: 2.8106
Epoch 8/10
50/50 [==============================] - 0s 326us/step - loss: 2.8076
Epoch 9/10
50/50 [==============================] - 0s 362us/step - loss: 2.8043
Epoch 10/10
50/50 [==============================] - 0s 352us/step - loss: 2.8012


## Part 5: Memory-friendly Data Generation

- Here, we modify the data generation function of skip-gram
- `yield`: it will return generators. And generators do not store all the values in memory. It will return value during each iteration.

sample code
```
generator = (x * x for x in range(3))
for i in generator:
    print(i)
```

https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do


In [29]:
def generate_data_live(corpus, window_size, V):
    """
    corpus is the list of sequence of words index
    window_size is used to define  
    V is the vocab Size
    """
    maxlen = window_size*2
    for words in corpus:
        labels   = []
        in_words = [] 
        L = len(words)
        for index, word in enumerate(words):
            s = index - window_size
            e = index + window_size + 1
            for i in range(s, e):
                if 0<= i < L and i != index:
                    in_words.append([word])
                    labels.append(words[i])
        x = np.array(in_words,dtype=np.int32)
        y = to_categorical(labels, V)
        yield (x, y) 


In [30]:
## here you should define skipgram from scratch

for ite in range(50):
    loss = 0.
    for x, y in generate_data_live(toy_seq_corpus, window_size, V):
        #updated parameters based on data samples provided without regard to any fixed batch size
        loss += skipgram.train_on_batch(x, y)
    print(ite, loss)

0 26.058088541030884
1 26.03108501434326
2 26.004350662231445
3 25.977773666381836
4 25.951301097869873
5 25.924898386001587
6 25.898550748825073
7 25.872254371643066
8 25.84601855278015
9 25.819857358932495
10 25.793790817260742
11 25.767837524414062
12 25.742018222808838
13 25.716354846954346
14 25.69086241722107
15 25.665558576583862
16 25.640456199645996
17 25.615569591522217
18 25.59090828895569
19 25.56648278236389
20 25.542301893234253
21 25.518372774124146
22 25.494699478149414
23 25.4712872505188
24 25.448142290115356
25 25.42526602745056
26 25.402658462524414
27 25.380325317382812
28 25.358264684677124
29 25.336475610733032
30 25.31495952606201
31 25.29371428489685
32 25.272737741470337
33 25.252029180526733
34 25.231585025787354
35 25.211401224136353
36 25.191476583480835
37 25.17180609703064
38 25.15238618850708
39 25.133212089538574
40 25.11427903175354
41 25.095585107803345
42 25.07712173461914
43 25.058887243270874
44 25.04087471961975
45 25.023080348968506
46 25.0054984